In [1]:
import dicom
import numpy as np
import os
import glob
import nibabel as nib

/usr/local/lib/python3.5/dist-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)
/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
ACDCimgpath='/opt/data/acdc/niftidata/'
ACDClblpath='/opt/output/acdc/norm/1/3/labels/'

In [3]:
ACDClbls=glob.glob(ACDClblpath)

In [ ]:
#Find Patients
results = []
for root, _, files in os.walk(ACDClblpath):
    rootnode = root.split("/")
    label = rootnode[-2]
    patient = rootnode[-1]
    patient_path = ACDClblpath + patient
    
    results.append(patientVolume(patient_path))
    
    
    #print ('rootnode', rootnode, label, patient, files, tempdir)

A slice has one frame
slice8 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice2 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice9 ['frame1']
A slice has one frame
slice10 ['frame1']
A slice has one frame
slice2 ['frame1']
A slice has one frame
slice14 ['frame1']
A slice has one frame
slice8 ['frame1']
A slice has one frame
slice14 ['frame1']
A slice has one frame
slice10 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice10 ['frame1']
A slice has one frame
slice9 ['frame1']
A slice has one frame
slice10 ['frame1']
A slice has one frame
slice10 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice11 ['frame1']
A slice has one frame
slice2 ['frame1']
A slice has one frame
slice1 ['frame1']
A slice has one frame
slice8 ['frame1']
A slice has one frame
slice8 ['frame1']
A slice has one frame
slice16 ['frame1']
A slice has one frame
slice1 ['

In [ ]:
results

In [16]:
def patientVolume(patient_path):
    for root,_, files in os.walk(patient_path):
        norm = None
        slicedict={}
        #slicenumbers=[]
        for i in files:
            filenode = i.split("_")
            #print (filenode)
            slicenum = filenode[1]
            frame = filenode[2]
            lblpath = root +'/'+ i
        
            #print (i, slicenum, frame)
            origframepath = getNiftiHeader(patient, slicenum, frame)

            origimg = nib.load(origframepath[2])
        
            if norm is not None:
                ps = origimg.header.get('pixdim')[1]
            else: 
                ps = 1
        
            thickness = origimg.header.get('pixdim')[3]
            lbl = np.load(lblpath)
        
            frac = getfrac(lbl)
            #area = frac * ps * len(lbl)
            area = getarea(lbl, frac, ps)
            volume = thickness * area
            #volume = (thickness/2) * area
        
            if slicenum in slicedict:
                slicedict[slicenum].update({frame:
                                            {'fraction': frac,
                                             'Area': area, 
                                             'Volume': volume}})
            else:
                slicedict[slicenum] = {frame:
                                    {'fraction': frac,
                                    'Area': area, 
                                    'Volume': volume}}
        
        
        #slicenumbers[slicenum]=([frame, frac, area, volume])
        #print ('rootnode', filenode,slicenum, frame, origframepath, frac, volume)
        #print (slicenum, frame, frac, volume)
        #return frac, area, volume
        ESF, ESV, EDF, EDV = getVolumes(slicedict)
        return (patient, ESV, EDV)

In [21]:
def getVolumes(slicedict):
    ESF = []
    ESVlist = []
    EDF = []
    EDVlist = []
    ESArea = []
    EDArea = []
    SL = []
    #print (slicedict)
    for key in slicedict:
        frames = list(slicedict[key].keys())
        #print (frames)
        if len(frames) > 2:
            print ('A slice has more than two frames')
        
        if len(frames) < 2:
            print ('A slice has one frame')
            print (key, frames)
            
        if len(frames) == 2:
            for i in range(len(frames)-1):
                #print (slicedict[key][frames[i]]['fraction'], slicedict[key][frames[i+1]]['fraction'] )
                if slicedict[key][frames[i]]['fraction'] < slicedict[key][frames[i+1]]['fraction']:
                    ESF.append( key +'_'+ frames[i])
                    ESVlist.append(slicedict[key][frames[i]]['Volume'])
                    EDF.append(key +'_'+ frames[i+1])
                    EDVlist.append(slicedict[key][frames[i+1]]['Volume'])
                else:
                    ESF.append(key +'_'+ frames[i+1])
                    ESVlist.append(slicedict[key][frames[i+1]]['Volume'])
                    EDF.append(key +'_' + frames [i])
                    EDVlist.append(slicedict[key][frames[i]]['Volume'])
    
    
    
    ESV = sum(ESVlist)
    EDV = sum(EDVlist)
    
    ESV = ESV / 1000
    EDV = EDV / 1000
    return ESF, ESV, EDF, EDV

In [10]:
def getNiftiHeader(patient, slicenum, framenum):
    imgpatientpath = ACDCimgpath+patient
    slicepath = imgpatientpath+'/'+patient+'_'+slicenum
    num = int(framenum.split('frame')[1])
    framepath = slicepath+'/'+patient+'_'+slicenum+'_frame'+str(num)+'.nii'
    return imgpatientpath, slicepath, framepath

In [11]:
def getfrac(img):
    frac = (sum(i == 1 for i in img.flatten()))/float(len(img.flatten()))
    return frac

In [12]:
def getarea(img, frac, ps): 
    #area = frac * ((sum(i ==1 for i in img.flatten())) * len(img))
    #area = frac * (ps * len(img))
    area = frac * (ps*len(img))**2
    return area